In [159]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, TensorDataset
import random
import numpy as np
from sklearn.decomposition import PCA
from typing import Tuple
from tqdm import tqdm
from scipy.spatial import distance_matrix
import pandas as pd

In [160]:
root = "/Users/kristiansjorslevnielsen/Documents/DVP7/Features/"
# load data
X_hpo = torch.tensor(np.load(root + "X_hpo_Img.npy" ))                       
y_hpo = np.load(root + "y_hpo_Img.npy" )

X_valid = torch.tensor( np.load(root + "X_val_Img.npy" ) )
y_valid = np.load(root + "y_val_Img.npy")

X_hpo_tensor = torch.tensor(X_hpo, dtype=torch.long)
y_hpo_tensor = torch.tensor(y_hpo, dtype=torch.long)

X_valid_tensor = torch.tensor(X_valid, dtype=torch.long)
y_valid_tensor = torch.tensor(y_valid, dtype=torch.long)

hpo_dataset = TensorDataset(X_hpo_tensor, y_hpo_tensor)
valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)

hpo_loader = DataLoader(hpo_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)


/var/folders/9k/3q09wnl130996d8g7j3bwgbr0000gn/T/ipykernel_98979/3317944195.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_hpo_tensor = torch.tensor(X_hpo, dtype=torch.long)
/var/folders/9k/3q09wnl130996d8g7j3bwgbr0000gn/T/ipykernel_98979/3317944195.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_valid_tensor = torch.tensor(X_valid, dtype=torch.long)


In [161]:
np.unique(y_valid)

array([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,   11,
         12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   22,
         23,   24,   25,   26,   27,   28,   29,   30,   31,   32,   33,
         34,   35,   36,   37,   38,   39,   40,   41,   42,   43,   44,
         45,   46,   47,   48,   49,   50,   51,   52,   53,   54,   55,
         56,   57,   58,   59,   60,   61,   62,   63,   64,   65,   66,
         67,   68,   69,   70,   71,   72,   73,   74,   75,   76,   77,
         78,   79,   80,   81,   82,   83,   84,   85,   86,   87,   88,
         89,   90,   91,   92,   93,   94,   95,   96,   97,   98,   99,
        100,  101,  102,  103,  104,  105,  106,  107,  108,  109,  110,
        111,  112,  113,  114,  115,  116,  117,  118,  119,  120,  121,
        122,  123,  124,  125,  126,  127,  128,  129,  130,  131,  132,
        133,  134,  135,  136,  137,  138,  139,  140,  141,  142,  143,
        144,  145,  146,  147,  148,  149,  150,  1

In [162]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
losses = []
maps = []

In [163]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SUBIC_encoder(nn.Module): 
    def __init__(self, input_size=4096, bits=48, num_classes=10, num_blocks=8, block_size=6):
        super(SUBIC_encoder, self).__init__()
       
        assert bits % num_blocks == 0, "Bits must be divisible by num_blocks"

        self.input_size = input_size
        self.bits = bits 
        self.num_blocks = num_blocks
        self.block_size = block_size
        
        # Define the encoder structure
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 256), 
            nn.ReLU(),
            nn.Linear(256, bits)
        )  # Outputs binary feature vectors
        
        self.fc3 = nn.Linear(bits, num_classes)  # Logits for num_classes
    
    def block_softmax(self, x):
        
        batch_size = x.shape[0]
        block_size = x.shape[1] // self.num_blocks
        
        # Ensure that x has the expected shape
        assert x.shape[1] == self.bits, f"Expected shape [batch_size, {self.bits}], got {x.shape}"
        
        # Reshape and apply softmax
        x = x.view(batch_size, self.num_blocks, block_size)
        x = F.softmax(x, dim=-1) 
        return x.view(batch_size, -1) #-1 refers to the value that will match the original elements 
    
    def block_one_hot(self, x):
        batch_size = x.shape[0]

        x = x.view(batch_size, self.num_blocks, self.block_size)
        max_indices = x.argmax(dim=-1, keepdim=True)
        
        # Create one-hot encoding
        one_hot = torch.zeros_like(x).scatter_(-1, max_indices, 1)

        return one_hot.view(batch_size, self.bits)
    
    def forward(self, x, use_one_hot=False):
        # Ensure x is a flat tensor before passing to encoder
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)  # Flatten if necessary

        z = self.encoder(x)

        if use_one_hot:
            binary_codes = self.block_one_hot(z)
        else:
            binary_codes = self.block_softmax(z)

        class_probs = F.softmax(self.fc3(binary_codes), dim=-1) 

        return class_probs, binary_codes


In [164]:
from itertools import product # create cartesian products for all params

bits_12_param_grid = {
'bits':12,
'num_blocks':
[3, 4],
'gamma':[0.2, 0.4, 0.6, 0.8],
'mu':[0.2, 0.4, 0.6, 0.8],
'learning_rate': [0.0001, 0.001, 0.005, 0.01]
}

bits_24_param_grid = {
'bits': 24,
'num_blocks':
[3, 6, 12],
'gamma':[0.2, 0.4, 0.6, 0.8],
'mu':[0.2, 0.4, 0.6, 0.8],
'learning_rate': [0.0001, 0.001, 0.005, 0.01]
}

bits_32_param_grid = {
'bits':32,
'num_blocks':
[4, 8, 16],
'gamma':[0.2, 0.4, 0.6, 0.8],
'mu':[0.2, 0.4, 0.6, 0.8],
'learning_rate': [0.0001, 0.001, 0.005, 0.01]
}

bits_48_param_grid = {
'bits':48,
'num_blocks':
[6, 12, 24],
'gamma':[0.2, 0.4, 0.6, 0.8],
'mu':[0.2, 0.4, 0.6, 0.8],
'learning_rate': [0.0001, 0.001, 0.005, 0.01]
}

def generate_combinations(param_grid):
    keys = list(param_grid.keys())
    values = [v if isinstance(v, list) else [v] for v in param_grid.values()] # make list of dicts, if value is a list then take value from the list if not then take that scalar value
    gamma_idx = keys.index("gamma")
    mu_idx = keys.index("mu")
    gamma_mu_pairs = list(zip(values[gamma_idx], values[mu_idx]))
    combined_values = values[:gamma_idx] + [gamma_mu_pairs] + values[mu_idx+1:]
    combined_keys = keys[:gamma_idx] + ["gamma_mu"] + keys[mu_idx+1:]
    comb = list(product(*combined_values))
    param_combinations_dicts = [
        {
            **dict(zip(combined_keys, comb_item)),
            'gamma': comb_item[combined_keys.index('gamma_mu')][0],
            'mu': comb_item[combined_keys.index('gamma_mu')][1]
        } for comb_item in comb
    ]
    for comb_dict in param_combinations_dicts:
        del comb_dict['gamma_mu']
    
    return param_combinations_dicts

In [165]:
def one_hot_encode(a, m = 0):
    if isinstance(a, torch.Tensor):
         a = a.cpu().numpy()
    if m<a.max():
        m = a.max()
    b = np.zeros((a.size, m + 1))
    b[np.arange(a.size), a] = 1
    
    return b
def meanAveragePrecision(test_hashes, training_hashes, test_labels, training_labels, n_labels=0):
    aps = []
    if len(training_labels.shape) == 1:
        training_labels = one_hot_encode(training_labels, n_labels)
        test_labels = one_hot_encode(test_labels, n_labels)
    for i, test_hash in enumerate(tqdm(test_hashes)):
        label = test_labels[i]
        distances = np.abs(training_hashes - test_hashes[i]).sum(axis=1)
        tp = np.where((training_labels*label).sum(axis=1)>0, 1, 0)
        hash_df = pd.DataFrame({"distances":distances, "tp":tp}).reset_index()
        hash_df = hash_df.sort_values(["distances", "index"]).reset_index(drop=True)
        hash_df = hash_df.drop(["index", "distances"], axis=1).reset_index()
        hash_df = hash_df[hash_df["tp"]==1]
        hash_df["tp"] = hash_df["tp"].cumsum()
        hash_df["index"] = hash_df["index"] +1 
        precision = np.array(hash_df["tp"]) / np.array(hash_df["index"])
        ap = precision.mean()
        aps.append(ap)
    
    return np.array(aps).mean()

In [166]:
def entropy(p):
    entropy_result = -torch.sum(p * torch.log2(p + 1e-30), dim=-1)
    return entropy_result

def cross_entropy(class_prob, target):
    if len(target.shape) >1:
        s = (class_prob*target).mean(axis=1)
    else:
        s = class_prob[torch.arange(len(target)), target]
    return -torch.log2(s)/torch.log2(torch.FloatTensor([class_prob.shape[1]]))

def compute_total_loss(class_probs, target, binary_codes, num_blocks, block_size, gamma=0.5, mu=0.5):
    """
    Computes the total loss, which includes:
    - Cross-entropy classification loss
    - Mean entropy loss (encouraging one-hot encoding within each block)
    - Batch entropy loss (encouraging uniform distribution across blocks)
    
    Parameters:
    - class_probs: The class probabilities from the classification layer.
    - target: The true labels.
    - binary_codes: The binary codes generated by the encoder.
    - num_blocks: The number of blocks in the binary codes.
    - block_size: The size of each block in the binary codes.
    - gamma: Weight for the mean entropy loss.
    - mu: Weight for the batch entropy loss.
    
    """

    classification_loss = cross_entropy(class_probs, target)

    batch_size = binary_codes.shape[0]
    binary_codes = binary_codes.view(batch_size, num_blocks, block_size) #used in structure encoding

    #Mean Entropy Loss (encourages each block to resemble a one-hot vector) using softmax binary code
    mean_entropy_loss = entropy(binary_codes).mean(dim=1)

    #Batch Entropy Loss (encourages uniform distribution across blocks)
    average_support = binary_codes.mean(dim=0)  
    batch_entropy_loss = entropy(average_support).mean(dim=0)

    #Combine losses with weights gamma and mu
    entropy_loss = (gamma * mean_entropy_loss - mu * batch_entropy_loss)/torch.log2(torch.FloatTensor([block_size]))
    total_loss = (classification_loss + entropy_loss).mean()
    
    
    return total_loss

#logits, binary_codes = model(X_train, use_one_hot=False)
#loss = compute_total_loss(logits, y_train_tensor, binary_codes, num_blocks=8, block_size=4, gamma=0.5, mu=0.05)

In [167]:
def loss_function(epochs, bits, num_blocks, block_size, gamma, mu, learning_rate):
    # Initialize model
    model = SUBIC_encoder(
        bits=bits,
        input_size=X_hpo.shape[1],
        num_classes=1001,
        num_blocks=num_blocks,
        block_size=block_size
    )
    model.to(device)  # Move model to the specified device
    
    # Initialize optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Track losses
    losses = []

    for epoch in range(epochs):
        model.train()  # Set model to training mode
        total_loss = 0.0

        for images, labels in hpo_loader:
            images, labels = images.to(device), labels.to(device)
            images, labels = images.to(torch.float), labels.to(torch.long)

            # Forward pass
            class_probs, binary_codes = model(images, use_one_hot=False)

            # Compute loss
            loss = compute_total_loss(
                class_probs, labels, binary_codes, num_blocks, block_size, gamma, mu
            )

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # Accumulate loss
            total_loss += loss.item()

        # Compute average loss for the epoch
        avg_loss = total_loss / len(hpo_loader)
        losses.append(avg_loss)
        #print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")

    return model, losses

def map_on_call(model):
    
    model.eval()
    all_query_codes, all_query_labels = [], []
    all_db_codes, all_db_labels = [], []

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            images, labels = images.to(torch.float), labels.to(torch.long)

            _, binary_codes = model(images, use_one_hot=True)

            # Ensure binary_codes is a tensor
            if not isinstance(binary_codes, torch.Tensor):
                raise TypeError("Expected binary_codes to be a tensor.")
            
            all_db_codes.append(binary_codes)
            all_db_labels.append(labels)
            if len(all_query_codes) == 0:  
                all_query_codes.append(binary_codes.clone())  
                all_query_labels.append(labels.clone())

    # Concatenate all tensors
    all_query_codes = torch.cat(all_query_codes, dim=0)
    all_query_labels = torch.cat(all_query_labels, dim=0)
    all_db_codes = torch.cat(all_db_codes, dim=0)
    all_db_labels = torch.cat(all_db_labels, dim=0)

    # Calculate MAP Score
    map_score = meanAveragePrecision(
        all_query_codes,
        all_db_codes,
        all_query_labels,
        all_db_labels,
        n_labels=1000
        )

    return map_score 
        

In [168]:
bits_12_param_dicts = generate_combinations(bits_12_param_grid) 
bits_24_param_dicts = generate_combinations(bits_24_param_grid) 
bits_32_param_dicts = generate_combinations(bits_32_param_grid) 
bits_48_param_dicts = generate_combinations(bits_48_param_grid) 

In [169]:
results = []
epochs = 100
map_results = {}

# iterate over each combination, train the model and evaluate its performance 
# initailize model with current hyperparameters

def hyper_tuning(param_grid, epochs, hpo_train, hpo_loader, device):
    results = {}  # To store MAP scores for each hyperparameter set
    best_map_score = 0  # Initialize with a very low value
    best_params = None  # To store the best parameter set

    for params in param_grid:
        # Extract parameters from the current set
        bits = params['bits']
        num_blocks = params['num_blocks']
        block_size = bits // num_blocks
        gamma = params['gamma']
        mu = params['mu']
        learning_rate = params['learning_rate']

        print(f"Testing hyperparameters: {params}")

        # Initialize model with the current hyperparameters
        model = SUBIC_encoder(bits=bits, input_size= hpo_train.shape[1], num_classes=1001, num_blocks=num_blocks, block_size=block_size)
        model.to(device) 

        # Train the model with the current parameter set
        trained_model, _ = loss_function(epochs, bits, num_blocks, block_size, gamma, mu, learning_rate)

        # Evaluate the model using MAP score
        map_score = map_on_call(trained_model)
        print(f"MAP score: {map_score:.4f}")

        # Store the MAP score for the current parameter set
        results[tuple(params.items())] = map_score

        if map_score > best_map_score:
            best_map_score = map_score
            best_params = params
            
    return best_params, best_map_score, results




In [170]:
hyper_tuning(param_grid=bits_12_param_dicts, epochs=100, hpo_train=X_hpo, hpo_loader=hpo_loader, device=device)

Testing hyperparameters: {'bits': 12, 'num_blocks': 3, 'learning_rate': 0.0001, 'gamma': 0.2, 'mu': 0.2}


100%|██████████| 64/64 [00:00<00:00, 112.13it/s]


MAP score: 0.1697
Testing hyperparameters: {'bits': 12, 'num_blocks': 3, 'learning_rate': 0.001, 'gamma': 0.2, 'mu': 0.2}


100%|██████████| 64/64 [00:00<00:00, 118.94it/s]


MAP score: 0.1760
Testing hyperparameters: {'bits': 12, 'num_blocks': 3, 'learning_rate': 0.005, 'gamma': 0.2, 'mu': 0.2}


...

100%|██████████| 64/64 [00:00<00:00, 121.37it/s]


MAP score: 0.0912
Testing hyperparameters: {'bits': 12, 'num_blocks': 4, 'learning_rate': 0.01, 'gamma': 0.8, 'mu': 0.8}


100%|██████████| 64/64 [00:00<00:00, 120.47it/s]

MAP score: 0.0343


({'bits': 12,
  'num_blocks': 4,
  'learning_rate': 0.001,
  'gamma': 0.4,
  'mu': 0.4},
 0.18459626269569707,
 {(('bits', 12),
   ('num_blocks', 3),
   ('learning_rate', 0.0001),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.16971055206745572,
  (('bits', 12),
   ('num_blocks', 3),
   ('learning_rate', 0.001),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.17602711189750814,
  (('bits', 12),
   ('num_blocks', 3),
   ('learning_rate', 0.005),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.10692819174898456,
  (('bits', 12),
   ('num_blocks', 3),
   ('learning_rate', 0.01),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.017427405539185578,
  (('bits', 12),
   ('num_blocks', 3),
   ('learning_rate', 0.0001),
   ('gamma', 0.4),
   ('mu', 0.4)): 0.16995734769269732,
  (('bits', 12),
   ('num_blocks', 3),
   ('learning_rate', 0.001),
   ('gamma', 0.4),
   ('mu', 0.4)): 0.1645966604538863,
  (('bits', 12),
   ('num_blocks', 3),
   ('learning_rate', 0.005),
   ('gamma', 0.4),
   ('mu', 0.4)): 0.1459933404604717,
  (('bits', 12

In [171]:
hyper_tuning(param_grid=bits_24_param_dicts, epochs=100, hpo_train=X_hpo, hpo_loader=hpo_loader, device=device)

Testing hyperparameters: {'bits': 24, 'num_blocks': 3, 'learning_rate': 0.0001, 'gamma': 0.2, 'mu': 0.2}


100%|██████████| 64/64 [00:00<00:00, 125.53it/s]


MAP score: 0.2192
Testing hyperparameters: {'bits': 24, 'num_blocks': 3, 'learning_rate': 0.001, 'gamma': 0.2, 'mu': 0.2}


100%|██████████| 64/64 [00:00<00:00, 119.47it/s]


MAP score: 0.2315
Testing hyperparameters: {'bits': 24, 'num_blocks': 3, 'learning_rate': 0.005, 'gamma': 0.2, 'mu': 0.2}


...

100%|██████████| 64/64 [00:00<00:00, 115.21it/s]


MAP score: 0.1860
Testing hyperparameters: {'bits': 24, 'num_blocks': 12, 'learning_rate': 0.01, 'gamma': 0.8, 'mu': 0.8}


100%|██████████| 64/64 [00:00<00:00, 110.93it/s]

MAP score: 0.0283


({'bits': 24,
  'num_blocks': 6,
  'learning_rate': 0.001,
  'gamma': 0.2,
  'mu': 0.2},
 0.30478309700506656,
 {(('bits', 24),
   ('num_blocks', 3),
   ('learning_rate', 0.0001),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.21917243086577246,
  (('bits', 24),
   ('num_blocks', 3),
   ('learning_rate', 0.001),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.23147300693103615,
  (('bits', 24),
   ('num_blocks', 3),
   ('learning_rate', 0.005),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.15083377478705057,
  (('bits', 24),
   ('num_blocks', 3),
   ('learning_rate', 0.01),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.01728212538647435,
  (('bits', 24),
   ('num_blocks', 3),
   ('learning_rate', 0.0001),
   ('gamma', 0.4),
   ('mu', 0.4)): 0.22363319225146094,
  (('bits', 24),
   ('num_blocks', 3),
   ('learning_rate', 0.001),
   ('gamma', 0.4),
   ('mu', 0.4)): 0.23455978240490766,
  (('bits', 24),
   ('num_blocks', 3),
   ('learning_rate', 0.005),
   ('gamma', 0.4),
   ('mu', 0.4)): 0.1292612413664621,
  (('bits', 24

In [172]:
hyper_tuning(param_grid=bits_32_param_dicts, epochs=100, hpo_train=X_hpo, hpo_loader=hpo_loader, device=device)

Testing hyperparameters: {'bits': 32, 'num_blocks': 4, 'learning_rate': 0.0001, 'gamma': 0.2, 'mu': 0.2}


100%|██████████| 64/64 [00:00<00:00, 113.67it/s]


MAP score: 0.2636
Testing hyperparameters: {'bits': 32, 'num_blocks': 4, 'learning_rate': 0.001, 'gamma': 0.2, 'mu': 0.2}


100%|██████████| 64/64 [00:00<00:00, 112.01it/s]


MAP score: 0.2730
Testing hyperparameters: {'bits': 32, 'num_blocks': 4, 'learning_rate': 0.005, 'gamma': 0.2, 'mu': 0.2}


...

100%|██████████| 64/64 [00:00<00:00, 120.63it/s]


MAP score: 0.2163
Testing hyperparameters: {'bits': 32, 'num_blocks': 16, 'learning_rate': 0.01, 'gamma': 0.8, 'mu': 0.8}


100%|██████████| 64/64 [00:00<00:00, 121.71it/s]

MAP score: 0.1703


({'bits': 32,
  'num_blocks': 8,
  'learning_rate': 0.001,
  'gamma': 0.4,
  'mu': 0.4},
 0.3069988445021767,
 {(('bits', 32),
   ('num_blocks', 4),
   ('learning_rate', 0.0001),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.2635838576328709,
  (('bits', 32),
   ('num_blocks', 4),
   ('learning_rate', 0.001),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.2730196758822694,
  (('bits', 32),
   ('num_blocks', 4),
   ('learning_rate', 0.005),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.17827219107759446,
  (('bits', 32),
   ('num_blocks', 4),
   ('learning_rate', 0.01),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.0179533992397006,
  (('bits', 32),
   ('num_blocks', 4),
   ('learning_rate', 0.0001),
   ('gamma', 0.4),
   ('mu', 0.4)): 0.24511459049371892,
  (('bits', 32),
   ('num_blocks', 4),
   ('learning_rate', 0.001),
   ('gamma', 0.4),
   ('mu', 0.4)): 0.256180671659857,
  (('bits', 32),
   ('num_blocks', 4),
   ('learning_rate', 0.005),
   ('gamma', 0.4),
   ('mu', 0.4)): 0.17457291140268835,
  (('bits', 32),
  

In [173]:
hyper_tuning(param_grid=bits_48_param_dicts, epochs=100, hpo_train=X_hpo, hpo_loader=hpo_loader, device=device)

Testing hyperparameters: {'bits': 48, 'num_blocks': 6, 'learning_rate': 0.0001, 'gamma': 0.2, 'mu': 0.2}


100%|██████████| 64/64 [00:00<00:00, 123.69it/s]


MAP score: 0.3057
Testing hyperparameters: {'bits': 48, 'num_blocks': 6, 'learning_rate': 0.001, 'gamma': 0.2, 'mu': 0.2}


100%|██████████| 64/64 [00:00<00:00, 119.86it/s]


MAP score: 0.2884
Testing hyperparameters: {'bits': 48, 'num_blocks': 6, 'learning_rate': 0.005, 'gamma': 0.2, 'mu': 0.2}


...

100%|██████████| 64/64 [00:00<00:00, 119.73it/s]


MAP score: 0.2294
Testing hyperparameters: {'bits': 48, 'num_blocks': 24, 'learning_rate': 0.01, 'gamma': 0.8, 'mu': 0.8}


100%|██████████| 64/64 [00:00<00:00, 119.34it/s]

MAP score: 0.1068


({'bits': 48,
  'num_blocks': 24,
  'learning_rate': 0.001,
  'gamma': 0.8,
  'mu': 0.8},
 0.3356484951088116,
 {(('bits', 48),
   ('num_blocks', 6),
   ('learning_rate', 0.0001),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.3056907009005908,
  (('bits', 48),
   ('num_blocks', 6),
   ('learning_rate', 0.001),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.28839512361534514,
  (('bits', 48),
   ('num_blocks', 6),
   ('learning_rate', 0.005),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.18755171010381183,
  (('bits', 48),
   ('num_blocks', 6),
   ('learning_rate', 0.01),
   ('gamma', 0.2),
   ('mu', 0.2)): 0.0181567658411289,
  (('bits', 48),
   ('num_blocks', 6),
   ('learning_rate', 0.0001),
   ('gamma', 0.4),
   ('mu', 0.4)): 0.2648228662866433,
  (('bits', 48),
   ('num_blocks', 6),
   ('learning_rate', 0.001),
   ('gamma', 0.4),
   ('mu', 0.4)): 0.3096556062888196,
  (('bits', 48),
   ('num_blocks', 6),
   ('learning_rate', 0.005),
   ('gamma', 0.4),
   ('mu', 0.4)): 0.18640424951834092,
  (('bits', 48),
